# SETUP

In [1]:
import warnings
import numpy as np
import pandas as pd
import yfinance as yf
import sklearn
import seaborn  as sns            # plots
import matplotlib.pyplot as plt   # plots

# PARAMETERS

In [2]:
df_nasdaq = pd.read_csv("Nasdaq_tickers.csv")
nasdaq_tickers = df_nasdaq["Symbol"].tolist()
# print(nasdaq_tickers)

In [3]:
#  SET ticker, yf_period and yf_interval
#  ticker - securities to download

ticker = nasdaq_tickers

#  set market indexes to compare equities with
market_index = ['^DJI', '^IXIC', '^GSPC']  # Dow Jones, Nasdaq and S&P500

#  time period and interval
yf_period   = "20y"   # 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
yf_interval = "1d"    # 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo

#  print set parameters
print('TICKER:\t\t{}'.format(ticker))
print('MARKET INDEX:\t{}'.format(market_index))
print('PERIOD:\t\t{}'.format(yf_period))
print('INTERVAL:\t{}'.format(yf_interval))

TICKER:		['A', 'AA', 'AAC', 'AACG', 'AACI', 'AACIW', 'AADI', 'AAIC', 'AAIC^B', 'AAIC^C', 'AAL', 'AAM^A', 'AAM^B', 'AAMC', 'AAME', 'AAN', 'AAOI', 'AAON', 'AAP', 'AAPL', 'AAQC', 'AAT', 'AATC', 'AAU', 'AAWW', 'AB', 'ABB', 'ABBV', 'ABC', 'ABCB', 'ABCL', 'ABCM', 'ABEO', 'ABEV', 'ABG', 'ABGI', 'ABIO', 'ABM', 'ABMD', 'ABNB', 'ABOS', 'ABR', 'ABR^D', 'ABR^E', 'ABR^F', 'ABSI', 'ABST', 'ABT', 'ABUS', 'ABVC', 'AC', 'ACA', 'ACAC', 'ACAD', 'ACAH', 'ACAQ', 'ACB', 'ACBA', 'ACBAW', 'ACCD', 'ACCO', 'ACDC', 'ACDCW', 'ACEL', 'ACER', 'ACET', 'ACEV', 'ACEVU', 'ACEVW', 'ACGL', 'ACGLN', 'ACGLO', 'ACHC', 'ACHL', 'ACHR', 'ACHV', 'ACI', 'ACII', 'ACIU', 'ACIW', 'ACLS', 'ACLX', 'ACM', 'ACMR', 'ACN', 'ACNB', 'ACNT', 'ACON', 'ACONW', 'ACOR', 'ACP', 'ACP^A', 'ACQR', 'ACQRU', 'ACQRW', 'ACR', 'ACR^C', 'ACR^D', 'ACRE', 'ACRO', 'ACRS', 'ACRX', 'ACST', 'ACT', 'ACTG', 'ACU', 'ACV', 'ACVA', 'ACXP', 'ADAG', 'ADAL', 'ADAP', 'ADBE', 'ADC', 'ADC^A', 'ADCT', 'ADD', 'ADEA', 'ADER', 'ADERU', 'ADERW', 'ADES', 'ADEX', 'ADI', 'ADIL',

In [6]:
# #  1.  uppercase and sort ticker
# ticker = [x.upper() for x in ticker]
# ticker.sort()


# #  2.  set market indexes
# market_index = ['^DJI', '^IXIC', '^GSPC']  # Dow Jones, Nasdaq and S&P500


# #  3.  remove markets from ticker for plots, returns
# for market in market_index:
#     if market in ticker:
#         ticker.remove(market)


# #  4.  symbols = market_indexes + ticker
# symbols = []   # initialize list
symbols = ticker


# #  print set parameters
# print('\nSYMBOLS (ticker + market index):\n{}'.format(symbols))

In [7]:
#  set parameters to download
fin_info = ["shortName", "marketCap", "volume", "dividendRate"]

#  create dataframe
yf_info = pd.DataFrame(index = fin_info, columns = symbols)

for i in symbols:
    l = []             # initialize
    x = yf.Ticker(i)   # get ticker info
    for j in fin_info:
        if 'date' in j.lower():
            d = pd.to_datetime(x.info[j])
            if d is not None:
                l.append(d.strftime("%Y-%m-%d"))  # format date
        else:
            try:      # some parameters error
                l.append(x.info[j])
            except:   # ignore error and continue
                l.append("")
    yf_info[i] = l
    print('{}\t- financial information downloaded'.format(i))


yf_info

A	- financial information downloaded
AA	- financial information downloaded
AAC	- financial information downloaded
AACG	- financial information downloaded
AACI	- financial information downloaded
AACIW	- financial information downloaded
AADI	- financial information downloaded
AAIC	- financial information downloaded
AAIC^B	- financial information downloaded
AAIC^C	- financial information downloaded
AAL	- financial information downloaded
AAM^A	- financial information downloaded
AAM^B	- financial information downloaded
AAMC	- financial information downloaded
AAME	- financial information downloaded
AAN	- financial information downloaded
AAOI	- financial information downloaded
AAON	- financial information downloaded
AAP	- financial information downloaded
AAPL	- financial information downloaded
AAQC	- financial information downloaded
AAT	- financial information downloaded
AATC	- financial information downloaded
AAU	- financial information downloaded
AAWW	- financial information downloaded
AB	-

In [ ]:
warnings.filterwarnings("ignore")
pd.options.display.float_format = '{:.4%}'.format

# Date range
start = '2016-01-01'
end = '2019-12-30'

# Tickers of assets
assets = ['JCI', 'TGT', 'CMCSA', 'CPB', 'MO', 'APA', 'MMC', 'JPM',
          'ZION', 'PSA', 'BAX', 'BMY', 'LUV', 'PCAR', 'TXT', 'TMO',
          'DE', 'MSFT', 'HPQ', 'SEE', 'VZ', 'CNP', 'NI', 'T', 'BA']
assets.sort()

# Downloading data
data = yf.download(start = start, end = end)
data = data.loc[:,('Adj Close', slice(None))]
data.columns = assets

TypeError: download() missing 1 required positional argument: 'tickers'

In [ ]:
# Calculating returns

Y = data[assets].pct_change().dropna()

display(Y.head())

,APA,BA,BAX,BMY,CMCSA,CNP,CPB,DE,HPQ,JCI,...,NI,PCAR,PSA,SEE,T,TGT,TMO,TXT,VZ,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-05,-2.0257%,0.4057%,0.4036%,1.9693%,0.0180%,0.9305%,0.3678%,0.5783%,0.9483%,-1.1953%,...,1.5881%,0.0212%,2.8236%,0.9758%,0.6987%,1.7539%,-0.1729%,0.2410%,1.3735%,-1.0857%
2016-01-06,-11.4863%,-1.5879%,0.2411%,-1.7557%,-0.7727%,-1.2473%,-0.1736%,-1.1239%,-3.5867%,-0.9551%,...,0.5547%,0.0212%,0.1592%,-1.5647%,0.3108%,-1.0155%,-0.7653%,-3.0048%,-0.9034%,-2.9144%
2016-01-07,-5.1389%,-4.1922%,-1.6573%,-2.7699%,-1.1047%,-1.9769%,-1.2206%,-0.8856%,-4.6058%,-2.5394%,...,-2.2066%,-3.0310%,-1.0410%,-3.1556%,-1.6148%,-0.2700%,-2.2844%,-2.0570%,-0.5492%,-3.0020%
2016-01-08,0.2736%,-2.2705%,-1.6037%,-2.5425%,0.1099%,-0.2241%,0.5706%,-1.6402%,-1.7642%,-0.1649%,...,-0.1539%,-1.1366%,-0.7308%,-0.1449%,0.0895%,-3.3838%,-0.1117%,-1.1387%,-0.9720%,-1.1254%
2016-01-11,-4.3383%,0.1692%,-1.6851%,-1.0215%,0.0914%,-1.1791%,0.5674%,0.5287%,0.6616%,0.0330%,...,1.6436%,0.0000%,0.9869%,-0.1450%,1.2224%,1.4570%,0.5366%,-0.4608%,0.5800%,-1.9919%
